<a href="https://colab.research.google.com/github/aziz122596/uzcosmos_1/blob/main/task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import inspect
from transformers import TrainingArguments
print(inspect.signature(TrainingArguments.__init__))



(self, output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: Optional[float] = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: Union[transformers.trainer_utils.SchedulerType, str] = 'linear', lr_scheduler_kwargs: Union[dict, str, NoneType] = <factory>, warmup_ratio: float = 0.0, warmup_ste

In [ ]:
!pip uninstall transformers
!pip install transformers


Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.11/dist-packages/transformers-4.51.3.dist-info/*
    /usr/local/lib/python3.11/dist-packages/transformers/*
Proceed (Y/n)? 

In [ ]:
!pip install transformers datasets torch scikit-learn accelerate matplotlib seaborn pandas

In [ ]:
import transformers
print(f"--- Версия Transformers, ЗАГРУЖЕННАЯ в сессию: {transformers.__version__} ---")

import datasets
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import torch
import os # Для создания папок

print("\n--- Шаг 1: Загрузка датасета SST-2 ---")
dataset = datasets.load_dataset("glue", "sst2")
print(dataset)

print("\n--- Шаг 2: Токенизация ---")
MODEL_NAME = "bert-base-uncased"
print(f"Загрузка токенизатора {MODEL_NAME}...")
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=512)

print("Токенизация данных...")
tokenized_datasets = dataset.map(tokenize_function, batched=True)

print("Обработка датасета...")
tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

# Разделяем на обучающую и валидационную выборки
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["validation"]
print("Данные подготовлены.")

print("\n--- Шаг 3: Загрузка модели ---")
print(f"Загрузка модели {MODEL_NAME} для классификации последовательностей...")
num_labels=2 для бинарной классификации (positive/negative)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Модель загружена и перемещена на устройство: {device}")

print("\n--- Шаг 4: Функция для вычисления метрик ---")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='binary')
    return {
        'accuracy': accuracy,
        'f1': f1,
    }
print("Функция compute_metrics определена.")

minimal_output_dir = './results_sst2_minimal_test'
os.makedirs(minimal_output_dir, exist_ok=True) # Создаем папку
print(f"Проверка минимального вызова TrainingArguments(output_dir='{minimal_output_dir}')...")
minimal_args_ok = False
try:
    minimal_args = TrainingArguments(output_dir=minimal_output_dir)
    print(">>> Минимальный вызов TrainingArguments УСПЕШЕН.")
    minimal_args_ok = True
except Exception as e:
    print(f">>> Минимальный вызов TrainingArguments НЕУДАЧЕН: {e}")
    print(">>> Если этот минимальный вызов не работает, проблема серьезная.")
    print(">>> Возможно, потребуется 'Сбросить настройки среды выполнения...'")

print("--- Конец диагностики ---\n")


print("\n--- Шаг 5: Настройка аргументов обучения ---")

if minimal_args_ok:
    try:
        training_args = TrainingArguments(
            output_dir='./results_sst2',
            num_train_epochs=3,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=64,
            warmup_steps=500,
            weight_decay=0.01,
            logging_dir='./logs_sst2',
            logging_strategy="steps",
            logging_steps=100,
            eval_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
            metric_for_best_model="f1",
            report_to="tensorboard",
            save_total_limit=2,
        )
        print("Объект TrainingArguments успешно создан.")

        print("\n--- Шаг 6: Создание объекта Trainer ---")
        trainer = Trainer(
            model=model,                         # Модель
            args=training_args,                  # Аргументы обучения
            train_dataset=train_dataset,         # Обучающий датасет
            eval_dataset=eval_dataset,           # Валидационный датасет
            compute_metrics=compute_metrics,     # Функция метрик
            tokenizer=tokenizer,                 # Токенизатор
        )
        print("Объект Trainer успешно создан.")

        print("\n--- Шаг 7: Запуск обучения ---")
        torch.cuda.empty_cache()
        train_result = trainer.train()
        print("Обучение завершено.")

        #метрики обучения
        metrics = train_result.metrics
        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)


        print("\n--- Шаг 8: Сохранение лучшей модели ---")
        best_model_path = './results_sst2/best_model'
        print(f"Сохранение лучшей модели (и токенизатора) в {best_model_path}...")
        trainer.save_model(best_model_path)
        tokenizer.save_pretrained(best_model_path)
        print("Лучшая модель и токенизатор сохранены.")

        print("\n--- Шаг 9: Оценка лучшей модели ---")
        eval_metrics = trainer.evaluate(eval_dataset=eval_dataset)
        print(f"Метрики на валидационном наборе:")
        trainer.log_metrics("eval", eval_metrics)
        trainer.save_metrics("eval", eval_metrics)


print("\n--- Выполнение скрипта завершено---")